In [1]:
import blpapi
import pandas as pd
import pyodbc 

cnxn = pyodbc.connect(
    r'DRIVER={ODBC Driver 17 for SQL Server};'
    r'SERVER=e360-db01;'
    r'DATABASE=Voltage;'
    r'Trusted_Connection=yes;'
)

In [2]:
def send_request_for_tenor(tenor, df):
    request = refDataService.createRequest("HistoricalDataRequest")
    
    request.append("securities", f"NG{tenor} COMB Comdty")
    print(f"NG{tenor} COMB Comdty")
    request.append("fields", "PX_LAST")  
#     request.append("fields", "TRADE_LAST_CLOSE_VOLUME")
#     request.append("fields", "OPEN_INT")      

    request.set("startDate", "20230905") 
    request.set("endDate", "20230914") 
    request.set("periodicitySelection", "DAILY")  

    session.sendRequest(request)

    while True:
        ev = session.nextEvent(500)
        if ev.eventType() == blpapi.Event.RESPONSE or ev.eventType() == blpapi.Event.PARTIAL_RESPONSE:
            for msg in ev:
                security_data = msg.getElement("securityData")
                field_data = security_data.getElement("fieldData")
                
                for i in range(field_data.numValues()):
                    daily_data = field_data.getValue(i)
                    date = daily_data.getElementAsDatetime("date")
                    px_last = daily_data.getElementAsFloat("PX_LAST")
#                     volume = daily_data.getElementAsFloat("TRADE_LAST_CLOSE_VOLUME")
#                     open_int = daily_data.getElementAsFloat("OPEN_INT")
                    
                    # Append data to DataFrame
                    df = df.append({
                        "Date": date,
                        "Tenor": f"NG{tenor} Comdty",
                        "PX_LAST": px_last
#                         "VOLUME": volume,
#                         "OPEN_INT": open_int
                    }, ignore_index=True)
            break
    return df

In [3]:
sessionOptions = blpapi.SessionOptions()
sessionOptions.setServerHost('localhost')
sessionOptions.setServerPort(8194)

session = blpapi.Session(sessionOptions)

if not session.start():
    print("Failed to start session.")
    exit()

if not session.openService("//blp/refdata"):
    print("Failed to open //blp/refdata")
    exit()

refDataService = session.getService("//blp/refdata")

In [4]:
# fieldInfoService = session.getService('//blp/apiflds')
# request = fieldInfoService.createRequest("FieldListRequest")
# request.append("fieldType", "All")
# request.set("returnFieldDocumentation", true)
# session.sendRequest(request)
# while True: 
#     ev = session.nextEvent(500)
#     for msg in ev:
#         msg.getElement('fieldData')
    

In [5]:
def format_df(df):
    search_cols = ['as_of_date', 'contract_month', 'expiry_date', 'tenor', 
                   'trade_date', 'option_strip', 'expiration_date', 'Date', 
                   'trading_date', 'start_date', 'end_date']
    for col in df.columns:
        if col in search_cols:
            df[col] = pd.to_datetime(df[col], format='%m/%d/%Y %X %p').dt.date
    return df

In [6]:
month_codes_q = "select * from month_code_meta where tenor between '10-01-2023' and '01-01-2026'"

month_codes = format_df(pd.read_sql(month_codes_q, cnxn))
month_codes

C:\Users\kjones\AppData\Local\Temp\ipykernel_28816\1254544172.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  month_codes = format_df(pd.read_sql(month_codes_q, cnxn))


,month_code,tenor,start_date,end_date
0,F24,2024-01-01,2024-01-01,2024-01-31
1,F25,2025-01-01,2025-01-01,2025-01-31
2,F26,2026-01-01,2026-01-01,2026-01-31
3,G24,2024-02-01,2024-02-01,2024-02-28
4,G25,2025-02-01,2025-02-01,2025-02-28
5,H24,2024-03-01,2024-03-01,2024-03-31
6,H25,2025-03-01,2025-03-01,2025-03-31
7,J24,2024-04-01,2024-04-01,2024-04-30
8,J25,2025-04-01,2025-04-01,2025-04-30
9,K24,2024-05-01,2024-05-01,2024-05-31


In [7]:
target_strikes_q = """
with futs as (select distinct as_of_date, tenor, price from exchange_prices 
where as_of_date between '09-05-2023' and '09-14-2023'
and exchange_code = 'PHH'
and tenor between '10-01-2023' and '01-01-2026'),
opts as (select distinct trade_date, option_strip, strike_price, put_or_call from ice_option_prices
where trade_date between '09-05-2023' and '09-14-2023'
and contract = 'PHE'
and abs(delta_factor) >= 0.2
and option_strip between '10-01-2023' and '01-01-2026'),
joined as (select * from opts inner join futs 
on opts.option_strip = futs.tenor 
and opts.trade_date = futs.as_of_date), 
final as (select trade_date, option_strip, strike_price, put_or_call, 
(price-strike_price) diff from joined 
where abs(price-strike_price) <= 1)
select distinct option_strip, strike_price, put_or_call from final
"""

target_strikes = format_df(pd.read_sql(target_strikes_q, cnxn))
target_strikes

C:\Users\kjones\AppData\Local\Temp\ipykernel_28816\1566415257.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  target_strikes = format_df(pd.read_sql(target_strikes_q, cnxn))


,option_strip,strike_price,put_or_call
0,2023-10-01,2.40,P
1,2023-10-01,2.90,P
2,2023-10-01,3.30,P
3,2023-10-01,3.60,P
4,2023-11-01,2.20,C
...,...,...,...
2239,2026-01-01,5.05,C
2240,2026-01-01,5.15,P
2241,2026-01-01,5.60,C
2242,2026-01-01,5.70,P


In [8]:
bbg_codes = {}

for code, tenor in zip(month_codes.month_code, month_codes.tenor):
    bbg_codes.update({''.join(list(code)[::2]) : tenor})
    
bbg_codes = pd.DataFrame.from_dict(bbg_codes, orient = 'index')\
            .reset_index().rename(columns = {'index' : 'bbg_code', 0 : 'tenor'})

bbg_codes

,bbg_code,tenor
0,F4,2024-01-01
1,F5,2025-01-01
2,F6,2026-01-01
3,G4,2024-02-01
4,G5,2025-02-01
5,H4,2024-03-01
6,H5,2025-03-01
7,J4,2024-04-01
8,J5,2025-04-01
9,K4,2024-05-01


In [9]:
# Initialize a new column 'full_bbg_code' in df2
target_strikes['full_bbg_code'] = ''

# Loop through df1 to find matches in df2
for index, row in bbg_codes.iterrows():
    tenor = row['tenor']
    mask = target_strikes['option_strip'] == tenor
    
    # Check if there are matches
    if mask.any():
        # Get the matching indices
        match_indices = target_strikes.index[mask].tolist()
        
        # Loop through matching indices and update 'full_bbg_code' for each match
        for match_index in match_indices:
            strike_price = '{:.2f}'.format(target_strikes.at[match_index, 'strike_price'])
            full_bbg_code = row['bbg_code'] + target_strikes.at[match_index, 'put_or_call'] + ' ' + strike_price
            target_strikes.at[match_index, 'full_bbg_code'] = full_bbg_code

# Display the updated df2
display(target_strikes)

,option_strip,strike_price,put_or_call,full_bbg_code
0,2023-10-01,2.40,P,V3P 2.40
1,2023-10-01,2.90,P,V3P 2.90
2,2023-10-01,3.30,P,V3P 3.30
3,2023-10-01,3.60,P,V3P 3.60
4,2023-11-01,2.20,C,X3C 2.20
...,...,...,...,...
2239,2026-01-01,5.05,C,F6C 5.05
2240,2026-01-01,5.15,P,F6P 5.15
2241,2026-01-01,5.60,C,F6C 5.60
2242,2026-01-01,5.70,P,F6P 5.70


In [10]:
# Initialize Empty DataFrame
df = pd.DataFrame(columns=["Date", "Tenor", "PX_LAST"])#, "VOLUME", "OPEN_INT"])

for tenor in target_strikes.full_bbg_code:
    df = send_request_for_tenor(tenor, df)
    break

session.stop()

df

NGV3P 2.40 COMB Comdty


AttributeError: 'DataFrame' object has no attribute 'append'

Below attempts to pull a pipeline flow  dataset from BBG

In [ ]:
import blpapi
import pandas as pd 

def initialize_session():
    # Create a Session
    sessionOptions = blpapi.SessionOptions()
    sessionOptions.setServerHost('localhost')
    sessionOptions.setServerPort(8194)
    
    session = blpapi.Session(sessionOptions)
    if not session.start():
        print("Failed to start session.")
        return None
    
    if not session.openService("//blp/refdata"):
        print("Failed to open //blp/refdata")
        return None
    
    return session

def fetch_dataset_data(session, dataset_id, parameters):
    refDataService = session.getService("//blp/refdata")
    request = refDataService.createRequest("HistoricalDataRequest")
    
    request.set("securities", dataset_id)
    
    for key, value in parameters.items():
        request.set(key, value)
    
    session.sendRequest(request)
    
    data = []
    while True:
        event = session.nextEvent(500)
        if event.eventType() == blpapi.Event.RESPONSE or \
           event.eventType() == blpapi.Event.PARTIAL_RESPONSE:
            for msg in event:
                data.append(msg)
        if event.eventType() == blpapi.Event.RESPONSE:
            break

    return data

# def fetch_dataset_data(session, dataset_id, parameters):
#     intraday_service = session.getService("//blp/intraday")
#     request = intraday_service.createRequest("IntradayTickRequest") #can also try IntradayBarRequest
    
#     request.set("security", dataset_id)
    
#     for key, value in parameters.items():
#         request.set(key, value)
    
#     session.sendRequest(request)
    
#     data = []
#     while True:
#         event = session.nextEvent(500)
#         if event.eventType() == blpapi.Event.RESPONSE or \
#            event.eventType() == blpapi.Event.PARTIAL_RESPONSE:
#             for msg in event:
#                 data.append(msg)
#         if event.eventType() == blpapi.Event.RESPONSE:
#             break

#     return data

# def fetch_dataset_data(session, dataset_id, parameters):
#     intraday_service = session.getService("//blp/intraday")
#     request = intraday_service.createRequest("IntradayTickRequest")
    
#     request.set("security", dataset_id)
    
#     for key, value in parameters.items():
#         request.set(key, value)
    
#     session.sendRequest(request)
    
#     # Create an empty list to store individual rows of data
#     rows = []
    
#     while True:
#         event = session.nextEvent(500)
#         if event.eventType() == blpapi.Event.RESPONSE or \
#            event.eventType() == blpapi.Event.PARTIAL_RESPONSE:
#             for msg in event:
#                 # Extract data from the message and append to rows
#                 for value in msg.getElement("tickData").getElement("tickData").values():
#                     row = {}
#                     for field in value.elements():
#                         row[field.name()] = field.getValue()
#                     rows.append(row)
                    
#         if event.eventType() == blpapi.Event.RESPONSE:
#             break

#     # Convert the list of rows to a DataFrame
#     df = pd.DataFrame(rows)
    
#     return df



if __name__ == "__main__":
    session = initialize_session()
    if session:
        dataset_id = "NGAS"  # Using the dataset identifier from Bloomberg
        parameters = {
            "startDate": "2023-01-01",
            "endDate": "2023-10-01",
            # Placeholder for other parameters
        }
        data = fetch_dataset_data(session, dataset_id, parameters)
        for msg in data:
            print(msg)
        session.stop()
